In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
import collections

In [2]:
data = pd.read_csv('data/results.csv')

In [14]:
'''
Total documents
Total word occurrences
Vocabulary size
Words occurring > 1000 times
Words occurring once
'''

total_documents = data.text.count()
# texts = ''.join(data.text).split()
texts = ''.join(data.text).lower()

In [35]:
# print(collections.Counter([x.lower() for x in ['a', 'b', 'c', 'a', 'b', 'b', 'A', 'B']]))
# pre process text

# REMOVER STOPWORDS?

regex = RegexpTokenizer(r'\w+')
tokens = regex.tokenize(texts)

In [36]:
word_counter = collections.Counter(tokens)
total_word_occurrences = sum(word_counter.values())
vocabulary_size = len(word_counter.keys())

In [52]:
most_frequent_words = len({x: word_counter[x] for x in word_counter if word_counter[x] > 1000})
less_frequent_words = len({x: word_counter[x] for x in word_counter if word_counter[x] == 1})

In [54]:
pd.DataFrame(data={'names':[
                            'Total Documents', 
                            'Total word ocurrences', 
                            'Vocabulary size', 
                            'Words occurring > 1000 times', 
                            'Words occurring once'
                            ], 
                   'values':[
                           total_documents, 
                           total_word_occurrences, 
                           vocabulary_size, 
                           most_frequent_words, 
                           less_frequent_words
                           ]
                  })

,names,values
0,Total Documents,249
1,Total word ocurrences,216704
2,Vocabulary size,22995
3,Words occurring > 1000 times,23
4,Words occurring once,11215
